In [1]:
# Aim- To define and extract Diwali and Non Diwali days data;
## and to find the hourly(3hr combined) Diwali day and Non-Diwali day concentration ratio of each city.

###  Importing Libraries

In [2]:
import pandas as pd
import numpy as np

###  Data loading

In [3]:
data=pd.read_excel('AndhraPradesh_CPCBData.xlsx',names=['DateTime','ToDate','PM','PM10','Temp','RH','WS','AT'], parse_dates=['DateTime'], na_values=['None'])

### Defining Index for data separation (each city)

In [4]:
index=np.where(np.array(data['DateTime']==' '))[0] # Index of rows where new city data begins.
l=np.array([len(data)],dtype=int) # Index of last row 
y=np.concatenate((index,l)) # Index of rows from where data will start periodically and ends.

index=np.where(np.array(data['DateTime']==' '))[0]

### Extracting and Concatenating eachCity names and eachCity data in arranged form

In [5]:
city_names=np.array([])
for i in range(len(index)):
    temp=data['PM'][index[i]:index[i]+1].values # Temporary storing city name as array (dtype=object)
    city=np.array(temp,dtype=str) # Converting above name as string.
    city=np.char.replace(city," ","") # Removing empty spaces
    city=np.char.replace(city,"-","") # Removing char (-).
    city_names=np.concatenate((city_names,city)) # Concatenate all cities name one by one.

In [6]:
city_dflist=[] # Trying to store all cities data as list

temp=pd.DataFrame([])
for i in range(len(y)):
    if i>(len(y)-2): #Stopping loop after no of cities name are finished.
        break
    globals()[f'df_{city_names[i]}']=data.iloc[y[i]:y[i+1]][2:]# [2:]- To ignore first(city name) and second (variables) columns.
    city_dflist+=[f'df_{city_names[i]}'] # It is coming as concat of city names hence string therefore apostrophe is there.

In [7]:
city_df=[df_SecretariatAmaravatiAPPCB,df_GulzarpetAnantapurAPPCB, df_TirumalaTirupatiAPPCB,df_PWDGroundsVijayawadaAPPCB,df_GVMCorporationVisakhapatnamAPPCB]

### Labelling Date, Hour, Day and 

In [8]:
for i  in city_df:
    i['DateTime'] = pd.to_datetime(i['DateTime'],format='%d-%m-%Y %H:%M')
    i['Hour']=i['DateTime'].dt.hour
    i['Day']=i['DateTime'].dt.day

### Defining Diwali(D) and NonDiwali(ND) days, calculating ratio of D days and ND days

In [9]:
Data_Ratio=pd.DataFrame([])
for i in range(len(city_df)):
# Pre Diwali day is with 24 hour gap from Diwali day on prior side.
    PreDiwali = city_df[i][(city_df[i]['DateTime'] > '2022-10-22 11:00:00') & 
       (city_df[i]['DateTime'] < '2022-10-23 12:00:00')].reset_index()
# Post Diwali day is with 24 hour gap from Diwali day on after side.
    PostDiwali = city_df[i][(city_df[i]['DateTime'] > '2022-10-26 11:00:00') &
       (city_df[i]['DateTime'] < '2022-10-27 12:00:00')].reset_index()
# Diwali day is from 12 p.m. of actual day to 12 p.m. of next day
    Diwali = city_df[i][(city_df[i]['DateTime'] > '2022-10-24 11:00:00') &
       (city_df[i]['DateTime'] < '2022-10-25 12:00:00')].reset_index()
    
    Diwali['RollingMean'] = Diwali['PM'].rolling(3).mean() # Taking 3 hour combined mean
    Diwali=Diwali[['Hour','PM','RollingMean']].add_suffix('_D')
    
    NonDiwali=pd.concat([PreDiwali,PostDiwali],axis=1, ignore_index=False) # NonDiwali day = PreDiwaliDay + Post DD
    NonDiwali['PM_mean']=NonDiwali['PM'].mean(axis=1)
    NonDiwali['RollingMean'] = NonDiwali['PM_mean'].rolling(3).mean()  # Taking 3 hour combined mean
    NonDiwali=NonDiwali[['Hour','PM','RollingMean']].add_suffix('_ND')
    
    Final=pd.concat([Diwali, NonDiwali], axis=1 ) 
    Final['Ratio']=Final['RollingMean_D']/Final['RollingMean_ND'] # Taking ratio of Diwali and NonDiwali hourly (3hr combined) mean.
    
    Data_Ratio[city_names[i]]=Final['Ratio'] # Saving Ratio of each city "city_names(i)" in Data_Ratio dataframe.
Data_Ratio['Hour']=Final['Hour_D']

In [10]:
Data_Ratio

,SecretariatAmaravatiAPPCB,GulzarpetAnantapurAPPCB,TirumalaTirupatiAPPCB,PWDGroundsVijayawadaAPPCB,GVMCorporationVisakhapatnamAPPCB,Hour
0,NaN,NaN,NaN,NaN,NaN,12
1,NaN,NaN,NaN,NaN,NaN,13
2,0.851995,0.789937,1.063218,NaN,1.364612,14
3,0.968162,0.785087,1.095373,NaN,1.064932,15
4,0.949388,0.760611,1.103318,NaN,1.009300,16
5,0.908400,0.764666,1.025641,NaN,1.008287,17
6,0.942911,0.769335,0.906934,NaN,1.070620,18
7,1.031873,1.041154,0.798333,NaN,1.708920,19
8,1.399085,2.970702,0.758405,NaN,4.033241,20
9,1.501894,5.487323,0.702336,NaN,7.546804,21
